In [1]:
"""
村上さんtoolbox
"""
# pandas 基礎集計クラス
import numpy as np
import pandas as pd
import copy
import seaborn as sns
from itertools import combinations

#http://qiita.com/tanemaki/items/2ed05e258ef4c9e6caac

# Jupyterで表示するためには、最初に以下を実行すること
%matplotlib inline

# Static Classで設計する
class pandas_tool:
    
    # All in one チェック（Jupyterのみ）
    def all_basic_summary_jupyter(df):
        print("■ 型の確認")
        display(pandas_tool.type(df))
        print("■ 数値型の確認")
        display(pandas_tool.summary(df)[0])
        print("■ カテゴリ型の確認")
        cate_var_data = list(df.select_dtypes(include=['object']).columns)
        ret = pandas_tool.freq(df,cate_var_data)
        for d in ret:
            display(pd.DataFrame(d))
            print("---------------")
        print("■ 欠損の確認")
        display(pandas_tool.check_missing(df))
    
    # 相関関係可視化（Jupyterのみ）
    def all_value_relation_visualize(df):
        #sns.set_context("poster", 1.2, {"lines.linewidth": 3})
        sns.pairplot(df,size=5)
    
    # カテゴリ変数でのヒートマップ（Jupyterのみ）
    def make_heatmap(df,x,y,value):
        target_df = df.pivot_table(index=x,values=value,columns=y)
        sns.heatmap(target_df, annot=True, fmt='1.1f', cmap='Blues')
    
    # 散布図（Jupyterのみ）
    def make_scatter_chart(df,x,y):
        #sns.jointplot(x=x, y=y, data=df, kind="hex")
        sns.jointplot(x=x, y=y, data=df)
    
    # 組み合わせでヒートマップを作成（Jupyterのみ）
    def all_make_heatmap(df,var_list,value):
        col_num = 2
        var_list_set = list(combinations(var_list,2))
        
        fig, axes = plt.subplots(int(len(var_list_set)/col_num)+1, col_num, figsize=(18,3+6.5*int(len(var_list_set)/col_num)))
        
        for i,target in enumerate(var_list_set):
            target_df = df.pivot_table(index=target[0],values=value,columns=target[1])
            sns.heatmap(target_df, annot=True, fmt='1.1f', cmap='Blues', ax=axes[int(i/col_num), i%col_num])
            
        plt.tight_layout()
    
    # 数値集計
    def summary(df,view=False):
        ret=df.describe()
        mis_ret=df.isnull().sum()
        if view:
            param=pd.get_option("display.max_columns")
            pd.set_option("display.max_columns",1000)
            print("・統計量")
            print(ret)
            print("・欠損値")
            print(mis_ret)
            pd.set_option("display.max_columns",param)
        return ret,mis_ret
    
    # 型チェック
    def type(df,view=False):
        ret = df.dtypes
        if view:
            param=pd.get_option("display.max_rows")
            pd.set_option("display.max_rows",1000)
            print(ret)
            pd.set_option("display.max_rows",param)
        return ret
    
    # 欠損チェック
    def check_missing(df,view=False):
        not_null_df=df.notnull()
        ret=pd.DataFrame()
        for name in not_null_df.columns:
            tmp_df=not_null_df[name].value_counts()
            tmp_df.name=name
            ret = pd.concat([ret,tmp_df],axis=1)
        
        if view:
            param=pd.get_option("display.max_columns")
            pd.set_option("display.max_columns",1000)
            print(ret)
            pd.set_option("display.max_columns",param)
        
        return ret
    
    # 欠損値のオブザベーションを抽出
    def get_miss_data(df,column,view=False):
        ret=df[df[column].isnull()]
        if view:
            param=pd.get_option("display.max_columns")
            pd.set_option("display.max_columns",1000)
            print(ret)
            pd.set_option("display.max_columns",param)
        return ret
    
    # 欠損値を中央値で補完
    def fill_miss_med(df,var_name):
        var=df[var_name].median()
        df[var_name].fillna(var,inplace=True)
        return df
    
    # 欠損値を0で補完
    def fill_miss_zero(df,var_name):
        df[var_name].fillna(0,inplace=True)
        return df
    
    # 特定の値を欠損とみなす
    def apply_miss_value(df,var_name,value):
        df[var_name]=df[var_name].replace(value,np.nan)
        return df
    
    # 重複チェック
    def check_dup(df,columns,view=False):
        ret=pd.DataFrame()
        for name in columns:
            dup_cnt=df[name].duplicated().sum()
            tmp_df = pd.DataFrame({'var_name':[name],'dup_cnt':[dup_cnt]})
            ret = pd.concat([ret,tmp_df],axis=0,ignore_index= True)
        
        if view:
            param=pd.get_option("display.max_columns")
            pd.set_option("display.max_columns",1000)
            print(ret)
            pd.set_option("display.max_columns",param)
        
        return ret
    
    # 組み合わせ重複チェック
    def check_dup_comb(df,columns,view=False):
        ret = df[columns].duplicated().sum()
        if view:
            param=pd.get_option("display.max_columns")
            pd.set_option("display.max_columns",1000)
            print(ret)
            pd.set_option("display.max_columns",param)
        
        return ret
    
    # ユニークデータ取得
    def get_uniq_data(df,uniq_key,sort_key,keep='first'):
        ret = df.sort_values(by=sort_key)
        ret.drop_duplicates(subset=uniq_key, keep=keep, inplace=True)
        return ret
    
    # カテゴリ集計
    def freq(df,columns,view=False):
        ret=list()
        for name in columns:
            tmp_df=df[name].value_counts()
            tmp_df.name=name
            #ret = pd.concat([ret,tmp_df],axis=1)
            ret.append(tmp_df)
        
        if view:
            param=pd.get_option("display.max_columns")
            pd.set_option("display.max_columns",1000)
            for r in ret:
                print(r)
                #display(r)
            pd.set_option("display.max_columns",param)
        
        return ret
    
    # 複雑な集計
    def tabulate(df,row,col=None,var='',func=np.sum,view=False):
        if var == '':
            tmp_df=df.reset_index(drop=False,inplace=False)
            ret=pd.pivot_table(data=tmp_df, values='index', index=row, columns=col, aggfunc='count', dropna=False, fill_value=0 ,margins = False)
            tmp_df=None
        else:
            ret=pd.pivot_table(data=df, values=var, index=row, columns=col, aggfunc=func, dropna=False, fill_value=0 ,margins = False)
        if view:
            param=pd.get_option("display.max_columns")
            pd.set_option("display.max_columns",1000)
            print(ret)
            pd.set_option("display.max_columns",param)
        
        return ret
    
    # マージ
    def merge(df1,df2,key,how,view=True):
        if view:
            print("df1のキー重複")
            pandas_tool.check_dup_comb(df1,key,True)
            print("df2のキー重複")
            pandas_tool.check_dup_comb(df2,key,True)
            
            print("df1のオブザベーション:{0}".format(len(df1)))
            print("df2のオブザベーション:{0}".format(len(df2)))
        
        ret=pd.merge(df1,df2,how=how,on=key)
        
        if view:
            print("mergeのオブザベーション:{0}".format(len(ret)))
        
        return ret
    
    # Rank
    def rank(df,var,num,suffix='_rank',check=False):
        labels=[i for i in range(0,num)]
        df[var+suffix]=pd.qcut(df[var], num, labels=labels)
        
        # check data
        if check:
            ret=pd.DataFrame()
            max_df=pandas_tool.tabulate(df=df,row=[var+suffix],var=var,func=np.max,view=False)
            max_df.name='max'
            min_df=pandas_tool.tabulate(df=df,row=[var+suffix],var=var,func=np.min,view=False)
            min_df.name='min'
            cnt_df=pandas_tool.tabulate(df=df,row=[var+suffix],var=var,func='count',view=False)
            cnt_df.name='count'
            ret=pd.concat([ret,min_df,max_df,cnt_df],axis=1)
            return df,ret
            
        return df
    
    # Rank適用(min基準)
    def apply_rank(df,rank_df):
        tmp_df=copy.deepcopy(rank_df)
        tmp_df.reset_index(drop=False,inplace=True)
        target_name=tmp_df.columns[3]
        tmp_df.columns=["rank","min","max","cnt"]
        
        def judge_thld(row):
            ret_var = -1
            cond_list = ["if 0 : ret_var = 0"]
            
            for i in range(1,len(tmp_df)):
                cond_list.append("elif row < " +str(tmp_df.ix[i,'min'])+ " : ret_var = " + str(tmp_df.ix[i-1,'rank']))
            
            cond_list.append("else: ret_var = " + str(tmp_df.ix[len(tmp_df)-1,'rank']))
            cond_str="\r\n".join(cond_list)
            # ローカル辞書をexecと共有する
            local_dict=locals()
            exec(cond_str,local_dict)
            return local_dict["ret_var"]
        
        df[target_name+"_rank"]=df[target_name].apply(judge_thld)
        return df
    
    # Min%以下はMin%点に、Max%以上はMax%点にクリップする
    def clip_min_max(df,col_list,apply_df=None,max_pct=0.99,min_pct=0.01):
        p_min = df[col_list].quantile(min_pct)
        p_max = df[col_list].quantile(max_pct)
        
        df[col] = df[col_list].clip(p_min,p_max,axis=1)
        
        # もしも適用先のデータがあるならば（例えば検証データ）対応
        if apply_df is not None:
            apply_df[col] = apply_df[col_list].clip(p_min,p_max,axis=1)
            return df,apply_df
        else:
            return df
    
    
    # 文字列→数値変換
    def conv_float(df,column,percent_flg=False):
        
        def conv_f(row):
            if row[column] == "" or row[column] is np.nan:
                return np.nan
            else:
                return float(row[column])
        
        df[column]=df[column].str.replace("\\","").str.replace(",","").str.replace("%","").str.strip()
        df[column]=df.apply(conv_f,axis=1)
        
        if percent_flg:
            df[column]=df[column]/100
        
        return df

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import japanize_matplotlib
import seaborn as sns

In [114]:
#data1は①をエクセルでコピペして切り出したもの。分野ごとの企画ペースと最終ペースが記載してある。
"""
ファイル内のすべてのシート名をループして、
各シートをデータフレームとしてdfsというリストの各要素に保存。
dfs[X]で任意のデータフレームとして機能。
"""
import pandas as pd
name_excel = "data1.xlsx"
path_folder = r"/Users/s.ogura/Documents/CRLEA/data/20220428"#Excelが置いてあるフォルダパス
path_file = r'{p}/{n}'.format(p = path_folder, n = name_excel)#ファイルパスとファイル名
input_file = pd.ExcelFile(path_file)
sn_list_data1 = input_file.sheet_names
# sn_list = sheet_names_list[0:len(sheet_names_list)-2]
#配列ループ
df = [] #空のリスト
for i in range(0,len(sn_list_data1)): #一覧と採点者を除外

    # Excelファイルの読み込み
    ret = pd.read_excel(path_file, sn_list_data1[i], index_col=None)
    #ret
    ret["採点回"]= sn_list_data1[i][0]#sn_listのiつ目の要素から1番目の文字
    ret["学年"]=int(sn_list_data1[i][1])#sn_listのiつ目の要素から2番目の文字    
    # Excel sheetsを空のリストの各要素に追加
    df.append(ret)

    
"""
集計したデータを縦に結合する。
df = df1.append(df2)
"""
dfs = df[0] #集計結果を一つのDFに結合
for i in df[1:]:
    dfs = dfs.append(i)
dfs = dfs.rename(columns={'科目分類名': '科目'})

map_dictionary ={"国語" : 10, "数学" :20, "英語" : 30} 
dfs['科目コード'] = dfs['科目'].map(map_dictionary) 
dfs = dfs[['科目','科目コード', '採点回', '学年', '割当','分野', '企画ペース', '最終ペース']]
dfs = dfs.rename(columns={'割当':'割当1'})
dfs['割当1'] = dfs.loc[:, "割当1"].astype('str')
df_data1 =dfs

In [220]:
#data2は②をエクセルでコピペして切り出したもの。分野ごとの企画ペースと最終ペースが記載してある。
"""
ファイル内のすべてのシート名をループして、
各シートをデータフレームとしてdfsというリストの各要素に保存。
dfs[X]で任意のデータフレームとして機能。
"""
# import pandas as pd
name_excel = "data2.xlsx"
path_folder = r"/Users/s.ogura/Documents/CRLEA/data/20220428"#Excelが置いてあるフォルダパス
path_file = r'{p}/{n}'.format(p = path_folder, n = name_excel)#ファイルパスとファイル名
input_file = pd.ExcelFile(path_file)
sn_list_data2 = input_file.sheet_names
sn_list_data2 = sn_list_data2[0:len(sn_list_data2)-4]
#配列ループ
df = [] #空のリスト
for i in range(0,3): #夏、秋、冬

    # Excelファイルの読み込み
    ret = pd.read_excel(path_file, sn_list_data2[i], index_col=None)
    #ret
    ret = ret.rename(columns={'採点回':'学年','割当単位':'割当'})
    ret["採点回"]= sn_list_data2[i][0]#sn_listのiつ目の要素から1番目の文字
    # ret["学年"]=int(sn_list_data1[i][1])#sn_listのiつ目の要素から2番目の文字    
    # Excel sheetsを空のリストの各要素に追加
    df.append(ret)

    
"""
集計したデータを縦に結合する。
df = df1.append(df2)
"""
dfs = df[0] #集計結果を一つのDFに結合
for i in df[1:]:
    dfs = dfs.append(i)

"""
作成したデータフレームの編集
・列名変更
・変数名変更
・行の削除
"""
#列名確認
# df.columns.values

dfs = dfs.replace({'学年': {'1年生':1,'2年生':2,'3年生':3,'客専':np.nan},
                   '割当': {'-':np.nan,'KS':np.nan,'パート分けナシ':np.nan,'?':np.nan},
                   '分野': {'-':np.nan,'KS':np.nan,'パート分けナシ':np.nan,'?':np.nan}})
dfs= dfs[~dfs["学年"].isna()]
dfs= dfs[~dfs["割当"].isna()]
dfs= dfs[~dfs["分野"].isna()]
dfs = dfs.rename(columns={'3始まり\nスタッフコード':'採点者コード'})
dfs = dfs[['採点者コード', '科目コード', '採点回', '学年', '割当','分野']]
dfs = dfs.rename(columns={'割当':'割当2'})
dfs['割当2'] = dfs.loc[:, "割当2"].astype('object')
df_data2 =dfs
df_data2.head()

,採点者コード,科目コード,採点回,学年,割当2,分野
0,3051286194,10,夏,1.0,01,評論
1,3051284876,10,夏,1.0,01,評論
3,3060309944,10,夏,1.0,01,評論
4,3060311559,10,夏,1.0,01,評論
5,3060320722,10,夏,1.0,01,評論


In [98]:
#分野マスターは分野の表記揺れを統一したもの。
"""
data1から分野を抜き出したcsvファイルを読み込む
"""

# import pandas as pd
name_csv = "bunyamaster_v2.csv"
path_folder = r"/Users/s.ogura/Documents/CRLEA/data/20220428"#Excelが置いてあるフォルダパス
path_file = r'{p}/{n}'.format(p = path_folder, n = name_csv)#ファイルパスとファイル名

# csvファイルの読み込みと空のリストに追加
bunya = pd.read_csv(filepath_or_buffer = path_file, sep=",")
bunya = bunya.rename(columns={'分野名':'分野','分野名（修正）':'分野_v2',})
bunya.head()

,科目コード,分野,分野_v2,ポイント採点,備考
0,10,評論,評論,0,NaN
1,10,現代文・評論,評論,0,NaN
2,10,現代文（評論）,評論,0,NaN
3,10,現代文・評論（理由説明）,評論,0,NaN
4,10,ポ評論,評論,1,NaN


In [91]:
#data3は③をリネームしたエクセル。採点コードごとのペースが記載してある。
"""
ファイル内のすべてのシート名をループして、各シートをデータフレームとしてdfというリストの各要素に保存。df[X]で任意のデータフレームとして機能。
"""
import pandas as pd
name_excel = "data3.xlsx"
path_folder = r"/Users/s.ogura/Documents/CRLEA/data/20220428"#Excelが置いてあるフォルダパス
path_file = r'{p}/{n}'.format(p = path_folder, n = name_excel)#ファイルパスとファイル名
input_file = pd.ExcelFile(path_file)
sn_list= input_file.sheet_names
sn_list = sn_list[0:len(sn_list)-2]
#配列ループ
df = [] #空のリスト
for i in range(0,len(sn_list)): 

    # Excelファイルの読み込み
    ret = pd.read_excel(path_file, sn_list[i], index_col=None)
    #ret
    ret["採点回"]= sn_list[i][0]#sn_listのiつ目の要素から1番目の文字
    ret["学年"]=int(sn_list[i][1])#sn_listのiつ目の要素から2番目の文字    
    # Excel sheetsを空のリストの各要素に追加
    df.append(ret)

    
"""
集計したデータを縦に結合する。
df = df1.append(df2)
"""
dfs = df[0] #集計結果を一つのDFに結合
for i in df[1:]:
    dfs = dfs.append(i)

# map_dictionary ={"国語" : 10, "数学" :20, "英語" : 30} 
# dfs['科目コード'] = dfs['科目'].map(map_dictionary) 
dfs = dfs[['科目コード','採点者コード', '採点回', '学年', '割当','ペース']]
dfs = dfs.rename(columns={'割当':'割当3'})
dfs=dfs[dfs['科目コード'].isin([10, 20,30])]#科目コード10、20、30を抽出
df_data3 = dfs
df_data3.head()

,科目コード,採点者コード,採点回,学年,割当3,ペース
0,10.0,3.050016e+09,夏,1,1.0,209.083637
1,10.0,3.050727e+09,夏,1,1.0,221.578735
2,30.0,3.050742e+09,夏,1,5.0,101.567398
3,10.0,3.051285e+09,夏,1,1.0,173.823108
4,10.0,3.051286e+09,夏,1,1.0,1649.259462


In [115]:
"""
Excelの書き出し data1
"""

name_excel_output = "data1_edited.xlsx"
path_folder = r"/Users/s.ogura/Documents/CRLEA/data/20220428"#Excelが置いてあるフォルダパス

with pd.ExcelWriter('{}/{}'.format(path_folder,name_excel_output)) as writer:
    df_data1.to_excel(writer, sheet_name='data1')

In [60]:
"""
Excelの書き出し data2
"""

name_excel_output = "data2_edited.xlsx"
path_folder = r"/Users/s.ogura/Documents/CRLEA/data/20220428"#Excelが置いてあるフォルダパス

with pd.ExcelWriter('{}/{}'.format(path_folder,name_excel_output)) as writer:
    df_data2.to_excel(writer, sheet_name='data2')

In [221]:
"""
2つのテーブルを結合させる
df_data2:採点者、科目、割当、採点回、学年、分野を記載したテーブル
bunya:分野の表記揺れをなおした分野_v2列がある
"""
df_data2b = pd.merge(df_data2,bunya, on = ["分野","科目コード"],how = 'left')
# df_data2b.drop(['氏名', '登録試験点数'], axis=1)
df_data2b.head()

,採点者コード,科目コード,採点回,学年,割当2,分野,分野_v2,ポイント採点,備考
0,3051286194,10,夏,1.0,01,評論,評論,0.0,NaN
1,3051284876,10,夏,1.0,01,評論,評論,0.0,NaN
2,3060309944,10,夏,1.0,01,評論,評論,0.0,NaN
3,3060311559,10,夏,1.0,01,評論,評論,0.0,NaN
4,3060320722,10,夏,1.0,01,評論,評論,0.0,NaN


In [222]:
"""
Excelの書き出し data2b
"""

name_excel_output = "data2b_edited.xlsx"
path_folder = r"/Users/s.ogura/Documents/CRLEA/data/20220428"#Excelが置いてあるフォルダパス

with pd.ExcelWriter('{}/{}'.format(path_folder,name_excel_output)) as writer:
    df_data2b.to_excel(writer, sheet_name='data2b')

In [120]:
"""
Excelの書き出し df2b_u
"""
df = df_data2b[["科目コード", "採点回","学年", "割当2"]].drop_duplicates()
name_excel_output = "df2b_u.xlsx"
path_folder = r"/Users/s.ogura/Documents/CRLEA/data/20220428"#Excelが置いてあるフォルダパス

with pd.ExcelWriter('{}/{}'.format(path_folder,name_excel_output)) as writer:
    df.to_excel(writer, sheet_name='temp')

In [92]:
"""
Excelの書き出し data3
"""

name_excel_output = "data3_edited.xlsx"
path_folder = r"/Users/s.ogura/Documents/CRLEA/data/20220428"#Excelが置いてあるフォルダパス

with pd.ExcelWriter('{}/{}'.format(path_folder,name_excel_output)) as writer:
    df_data3.to_excel(writer, sheet_name='data3')

In [294]:
#data1の割当を手作業で修正したもの。
"""
ファイル内のすべてのシート名をループして、
各シートをデータフレームとしてdfsというリストの各要素に保存。
dfs[X]で任意のデータフレームとして機能。
"""
name_excel = "data1_edited_manual.xlsx"
path_folder = r"/Users/s.ogura/Documents/CRLEA/data/20220428"#Excelが置いてあるフォルダパス
path_file = r'{p}/{n}'.format(p = path_folder, n = name_excel)#ファイルパスとファイル名
input_file = pd.ExcelFile(path_file)
sn_list = input_file.sheet_names
df = pd.read_excel(path_file, sn_list[0], index_col=0)
df = df.rename(columns={'割当1m':'割当'})
#学年のデータ型をintからobjectに
df["学年"] = df["学年"].astype('object')
df= df[~df["科目コード"].isna()]
df = df[["科目コード","採点回","学年",'割当','企画ペース', '最終ペース']]
# df['joint_key'] = df['科目コード'].astype(object) + 　df['採点回']+df['学年'].astype(object)+ df['割当']
df_1m =df

In [295]:
"""
Excelの書き出し df_1m
"""

name_excel_output = "df_1m.xlsx"
path_folder = r"/Users/s.ogura/Documents/CRLEA/data/20220428"#Excelが置いてあるフォルダパス

with pd.ExcelWriter('{}/{}'.format(path_folder,name_excel_output)) as writer:
    df_1m.to_excel(writer, sheet_name='temp')

In [251]:
"""
df_data2b:採点者コード、科目コード、採点回、学年、割当2、分野を記載したテーブル
df_data3:採点者コード、科目コード、採点回、学年、割当3、ペース
2つのテーブルを結合させる
科目コード、採点回、学年、割当をつなげた列を作成する
"""
df1 = df_data2b
df2 = df_data3
df = pandas_tool.merge(df1,df2,key=["採点者コード","科目コード","採点回","学年"],how = 'right',view=True)
df = df.rename(columns={'割当2':'割当'})
#学年のデータ型をintからobjectに
df["学年"] = df["学年"].astype('object')
# map_dictionary ={"1.0" : "1", "2.0" :"2", "3.0" : "3"} 
# df['学年'] = df['学年'].map(map_dictionary) 
df = df[["採点者コード","科目コード","採点回","学年",'割当','ペース', '分野', '分野_v2', 'ポイント採点', '備考']]
df_2b3 = df

df1のキー重複
0
df2のキー重複
0
df1のオブザベーション:15739
df2のオブザベーション:12376
mergeのオブザベーション:12376


In [252]:
"""
Excelの書き出し df_2b3
"""

name_excel_output = "df_2b3.xlsx"
path_folder = r"/Users/s.ogura/Documents/CRLEA/data/20220428"#Excelが置いてあるフォルダパス

with pd.ExcelWriter('{}/{}'.format(path_folder,name_excel_output)) as writer:
    df_2b3.to_excel(writer, sheet_name='temp')

In [208]:
#このセルは不要
"""
df_data2bm:採点者コード、科目コード、採点回、学年、、割当2、分野を記載したテーブル
df_data3:採点者コード、科目コード、採点回、学年、割当3、ペース
2つのテーブルを結合させる
科目コード、採点回、学年、割当をつなげた列を作成する
"""
#data2の割当を手作業で修正したもの。
"""
ファイル内のすべてのシート名をループして、
各シートをデータフレームとしてdfsというリストの各要素に保存。
dfs[X]で任意のデータフレームとして機能。
"""
name_excel = "data2b_edited_manual.xlsx"
path_folder = r"/Users/s.ogura/Documents/CRLEA/data/20220428"#Excelが置いてあるフォルダパス
path_file = r'{p}/{n}'.format(p = path_folder, n = name_excel)#ファイルパスとファイル名
input_file = pd.ExcelFile(path_file)
sn_list = input_file.sheet_names
df = pd.read_excel(path_file, sn_list[0], index_col=0)
df_2bm = df
df1 = df_2bm
df2 = df_data3
dfs = pandas_tool.merge(df1,df2,key=["採点者コード","科目コード","採点回","学年"],how = 'right',view=True)
dfs = dfs.rename(columns={'割当2':'割当'})
dfs_2bm3 = dfs

df1のキー重複
0
df2のキー重複
0
df1のオブザベーション:15868
df2のオブザベーション:12376
mergeのオブザベーション:12376


In [296]:
"""
df_2b3:採点者コード、科目コード、採点回、学年、割当、分野、ペース
df_1m:科目コード、採点回、学年、割当、企画ペース、最終ペース
"""
df1 = pd.read_excel(r"/Users/s.ogura/Documents/CRLEA/data/20220428/df_2b3.xlsx",
                    dtype = {'割当': object}, index_col=0)
df2 = pd.read_excel(r"/Users/s.ogura/Documents/CRLEA/data/20220428/df_1m.xlsx", index_col=0)

#科目コード、採点回、学年、割当をつなげた列を作成する

df1["temp1"] = df1['採点回'].astype(str).str.cat(df1['学年'].astype(str))
df1["temp2"] = df1['科目コード'].astype(str).str.cat(df1['割当'].astype(str))
df1["temp3"] = df1['temp1'].str.cat(df1['temp2'])

df2["temp1"] = df2['採点回'].astype(str).str.cat(df2['学年'].astype(str))
df2["temp2"] = df2['科目コード'].astype(str).str.cat(df2['割当'].astype(str))
df2["temp3"] = df2['temp1'].str.cat(df2['temp2'])

#重複する列を削除
df1 = df1[["temp3","採点者コード","ペース","分野","分野_v2","ポイント採点","備考"]]
df2 = df2[["temp3","企画ペース","最終ペース"]]
#2つのテーブルを結合させる
df = pandas_tool.merge(df1,df2,key=["temp3"],how = 'left',view=True)

df_1m2b3 = df

df1のキー重複
12127
df2のキー重複
1
df1のオブザベーション:12376
df2のオブザベーション:351
mergeのオブザベーション:12376


In [293]:
"""
Excelの書き出し df2
"""

name_excel_output = "df1_1m.xlsx"
path_folder = r"/Users/s.ogura/Documents/CRLEA/data/20220428"#Excelが置いてあるフォルダパス

with pd.ExcelWriter('{}/{}'.format(path_folder,name_excel_output)) as writer:
    df2.to_excel(writer, sheet_name='temp')

In [280]:
df1["temp3"]
df2["temp3"]

0         夏01101
1         夏01101
2         夏05301
3         夏01101
4         夏01101
          ...   
12371    冬058202
12372     冬06302
12373     冬03202
12374     冬09302
12375     冬01302
Name: temp3, Length: 12376, dtype: object

In [274]:
df1.dtypes
# df1.head()
# df_1m["学年"].unique()

採点者コード      int64
科目コード       int64
採点回        object
学年          int64
割当         object
ペース       float64
分野         object
分野_v2      object
ポイント採点    float64
備考         object
dtype: object

In [267]:
df2.dtypes
# df_2b3["学年"].unique()

科目コード      int64
採点回       object
学年         int64
割当        object
企画ペース    float64
最終ペース     object
dtype: object

In [291]:
df_1m2b3.head()

,temp3,採点者コード,ペース,分野,分野_v2,ポイント採点,備考,企画ペース,最終ペース
0,夏11001,3050015912,209.083637,評論,評論,0.0,NaN,NaN,NaN
1,夏11001,3050727415,221.578735,評論,評論,0.0,NaN,NaN,NaN
2,夏13005,3050742436,101.567398,英訳,和文英訳,0.0,ライティング,150.0,97.9
3,夏11001,3051284876,173.823108,評論,評論,0.0,NaN,NaN,NaN
4,夏11001,3051286194,1649.259462,評論,評論,0.0,NaN,NaN,NaN


In [297]:
"""
Excelの書き出し df_1m2b3
"""

name_excel_output = "df_1m2b3.xlsx"
path_folder = r"/Users/s.ogura/Documents/CRLEA/data/20220428"#Excelが置いてあるフォルダパス

with pd.ExcelWriter('{}/{}'.format(path_folder,name_excel_output)) as writer:
    df_1m2b3.to_excel(writer, sheet_name='temp')

In [178]:
df_1m['採点回'].str.cat(df_1m['割当'])
df_1m['学年'].astype(object)
df_1m['科目コード'].astype(object)

pandas.core.series.Series

In [158]:
# 型チェック
df_ret = pandas_tool.type(df_1m)
df_ret

科目        object
科目コード    float64
採点回       object
学年         int64
割当        object
企画ペース    float64
最終ペース     object
dtype: object

In [263]:
pd.merge(df1, df2, on=["科目コード","採点回","学年","割当"], how = "left")

,採点者コード,科目コード,採点回,学年,割当,ペース,分野,分野_v2,ポイント採点,備考,企画ペース,最終ペース
0,3.050016e+09,10.0,夏,1.0,01,209.083637,評論,評論,0.0,NaN,NaN,NaN
1,3.050727e+09,10.0,夏,1.0,01,221.578735,評論,評論,0.0,NaN,NaN,NaN
2,3.050742e+09,30.0,夏,1.0,05,101.567398,英訳,和文英訳,0.0,ライティング,150.0,97.9
3,3.051285e+09,10.0,夏,1.0,01,173.823108,評論,評論,0.0,NaN,NaN,NaN
4,3.051286e+09,10.0,夏,1.0,01,1649.259462,評論,評論,0.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
12371,3.070094e+09,20.0,冬,2.0,058,208.007777,指数対数,指数関数・対数関数,0.0,NaN,170.0,200.3
12372,3.070094e+09,30.0,冬,2.0,06,218.984890,0,NaN,NaN,NaN,250.0,168.3
12373,3.070094e+09,20.0,冬,2.0,03,211.585865,場合の数と確率,場合の数と確率,0.0,数学Ａ,180.0,146.6
12374,3.070094e+09,30.0,冬,2.0,09,40.996396,0,NaN,NaN,NaN,110.0,65.3


In [148]:
"""
Excelの書き出し dfs_1m2b3
"""

name_excel_output = "dfs_1m2b3.xlsx"
path_folder = r"/Users/s.ogura/Documents/CRLEA/data/20220428"#Excelが置いてあるフォルダパス

with pd.ExcelWriter('{}/{}'.format(path_folder,name_excel_output)) as writer:
    dfs_1m2b3.to_excel(writer, sheet_name='temp')

In [86]:
# df1 = df_data2b
# df2 = df_data3
# df4 = pandas_tool.merge(df1,df2,key=["採点者コード","科目コード","採点回","学年"],how = 'right',view=True)
# df_uniques = df4[["科目コード", "採点回","学年", "割当2"]].drop_duplicates()
# print(df_uniques)

       科目コード 採点回   学年  割当2
0       10.0   夏  1.0   01
2       30.0   夏  1.0   05
17      20.0   夏  1.0  034
22      10.0   夏  1.0   02
30      30.0   夏  1.0   01
...      ...  ..  ...  ...
10886   20.0   冬  2.0   06
11105   20.0   冬  2.0   08
11337   20.0   冬  2.0   09
11636   20.0   冬  2.0  NaN
11731   20.0   冬  2.0  107

[253 rows x 4 columns]


In [139]:
"""
Excelの書き出し df2b3
"""

name_excel_output = "df2b3.xlsx"
path_folder = r"/Users/s.ogura/Documents/CRLEA/data/20220428"#Excelが置いてあるフォルダパス

with pd.ExcelWriter('{}/{}'.format(path_folder,name_excel_output)) as writer:
    dfs_2b3.to_excel(writer, sheet_name='temp')

In [87]:
"""
Excelの書き出し df_unique
"""

name_excel_output = "df_temp_dfunique.xlsx"
path_folder = r"/Users/s.ogura/Documents/CRLEA/data/20220428"#Excelが置いてあるフォルダパス

with pd.ExcelWriter('{}/{}'.format(path_folder,name_excel_output)) as writer:
    df_uniques.to_excel(writer, sheet_name='temp')

In [16]:
"""
results2:科目、割当、採点回、学年毎の各集計値を記載したテーブル
bunya:科目、割当、採点回、学年に対応した分野と企画ペースを記載したテーブル
2つのテーブルを結合させる
"""
output = pd.merge(results2,bunya, on = ["科目コード","割当","採点回","学年"],how = 'outer')
output

,割当,科目コード,採点者数,所要合計時間,採点枚数合計,ペース平均,ペース最大値,ペース最小値,ペース中央値,ペース分散,ペース偏差二乗和,採点回,学年,分野,企画ペース,最終ペース,分野2
0,1.0,10.0,55.0,4963319.0,366913.0,349.615844,1649.259462,77.254963,309.220098,60788.556229,3.343371e+06,夏,1,評論,330.00,273.300,評論
1,1.0,20.0,107.0,16479859.0,405119.0,117.376035,400.000000,25.103984,109.744997,4439.792090,4.750578e+05,夏,1,ポ数と式,140.00,88.900,数と式
2,1.0,30.0,29.0,1881603.0,221741.0,744.867856,2367.953032,93.845825,616.084322,266023.340836,7.714677e+06,夏,1,英作,400.00,462.100,英作
3,2.0,10.0,59.0,10355031.0,369844.0,167.625006,349.726869,23.013071,178.527124,5702.131548,3.364258e+05,夏,1,ポ評論,205.00,133.100,評論
4,2.0,20.0,106.0,12295994.0,386688.0,149.573122,351.494467,11.650485,139.677414,6090.513035,6.455944e+05,夏,1,ポ数と式,140.00,112.900,数と式
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
553,12.0,54.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,冬,2,NaN,NaN,NaN,NaN
554,10.0,20.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,秋,2,ポ式と証明・高次方程式,175.00,208.400,式と証明・高次方程式
555,11.0,20.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,秋,2,"図形と方程式, 三角関数, 微分法, 数列, ベクトル",211.00,207.280,"図形と方程式, 三角関数, 微分法, 数列, ベクトル"
556,12.0,20.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,秋,2,ポ二次関数,400.00,416.100,二次関数
